# Facade

## O que é?

O padrao _Facade_ oferece uma interface simples e unificada para uma série de interfaces num sistema mais complexo.

## Por quê?

O objetivo principal do padrao _Facade_ é minimizar a complexidade de um sistema ao oferecer um método simplificado de acessar parte de sua funcionalidade, reduzindo o conhecimento necessário para usá-lo. Um exemplo clássico oferecido no livro é um compilador: embora o compilador seja um sistema complexo envolvendo múltiplas partes (scanner, parser, tokenizer, etc), o usuário final dificilmente precisará ter acesso a essas partes do sistema. Em lugar disso, usaria apenas o método _compile_.

## Estrutura

![struct](https://www.cs.unc.edu/~stotts/GOF/hires/Pictures/facade.gif)


## Exemplo

Roubemos o exemplo do _Builder_: uma classe responsável por criar um plano de dieta e exercício. 


In [2]:

from typing import List
import abc


class Exercise: 
    
    def __init__(self, name: str, reps: int):
        self.name = name
        self.reps = reps
        
    def __str__(self):
        return f"{self.name} (x{self.reps})"

    
class Meal: 
    
    def __init__(self, dish: str, quantity_in_grams: int):
        self.dish = dish
        self.quantity_in_grams = quantity_in_grams
        
    def __str__(self):
        return f"{self.dish} ({self.quantity_in_grams}g)"
    
    
class Day:
    
    def __init__(self, objective: str, exercises: List[Exercise]=[], meals: List[Meal]=[]):
        self.objective = objective
        self.exercises = exercises[::]
        self.meals = meals[::]
        
    def add_exercise(self, exercise: Exercise):
        self.exercises.append(exercise)
        
    def add_meal(self, meal: Meal):
        self.meals.append(meal)
    
    def __str__(self):
        exercise_str = (len(self.exercises) > 0)* '\nExercises: \n\t' + '\n\t'.join(map(str, self.exercises))
        meal_str = (len(self.meals) > 0)* '\nMeals: \n\t' + '\n\t'.join(map(str, self.meals))
        return f"Objective: {self.objective}{exercise_str}{meal_str}"

    
class Plan:
    
    def __init__(self, student: str, days: List[Day]=[]):
        self.days = days[::]
        self.student = student
        
    def add_day(self, day: Day):
        self.days.append(day)
    
    def __str__(self):
        return self.student + '\n\n' + '\n\n'.join(f"Day {i}: \n{str(day)}" for i, day in enumerate(self.days, 1))



    
# Builder abstrato

class PlanBuilder(abc.ABC):
    
    def add_day(self, objective: str) -> 'PlanBuilder':
        self.plan.add_day(Day(objective))
        return self

    def add_exercise(self, name: str, reps: int) -> 'PlanBuilder': 
        return self
    
    def add_meal(self, dish: str, quantity_in_grams: int) -> 'PlanBuilder': 
        return self
    
    @abc.abstractmethod
    def build(self) -> Plan:
        pass



class BodyBuildingPlanBuilder(PlanBuilder):
    
    def __init__(self, student: str):
        self.plan = Plan(student)
    
    def add_meal(self, dish: str, quantity_in_grams: int) -> 'PlanBuilder':
        self.current_day().add_meal(Meal(dish, quantity_in_grams))
        return self
    
    def add_exercise(self, name: str, reps: int) -> 'PlanBuilder':
        self.current_day().add_exercise(Exercise(name, reps))
        return self
        
    def current_day(self) -> Day:
        return self.plan.days[-1]
    
    def build(self) -> Plan:
        return self.plan



Para criar um plano de exercício + alimentacao, a interface funcionava assim:

In [3]:
body_building_plan_builder = BodyBuildingPlanBuilder('Gracyanne')

body_building_plan = (body_building_plan_builder.add_day('Legs')
           .add_meal('Sweet potato', 1000)
           .add_exercise('Leg press', 200)
       .add_day('Butt')
           .add_meal('Chicken breast', 1000)
           .add_exercise('Bulgarian squats', 200)
        .build()
       )

print(body_building_plan)

Gracyanne

Day 1: 
Objective: Legs
Exercises: 
	Leg press (x200)
Meals: 
	Sweet potato (1000g)

Day 2: 
Objective: Butt
Exercises: 
	Bulgarian squats (x200)
Meals: 
	Chicken breast (1000g)


Suponhamos que essa interface funciona bem para uso interno: os funcionários da academia sabem adicionar os dias, exercícios, refeicoes e dados do aluno corretamente. Mas imaginemos que a mesma academia queira disponibilizar essa interface para os alunos. Talvez alguns queiram planejar detalhadamente seus exercícios, mas a maioria preferiria evitar essa complexidade e optar pelo mesmo plano básico. O padrao _Facade_ pode ajudar nisso. Podemos implementá-lo com pequenas mudancas:

In [4]:
# Builder abstrato

class PlanBuilder(abc.ABC):

    @abc.abstractstaticmethod
    def create_default_plan(student: str) -> Plan:
        pass
    
    def add_day(self, objective: str) -> 'PlanBuilder':
        self.plan.add_day(Day(objective))
        return self

    def add_exercise(self, name: str, reps: int) -> 'PlanBuilder': 
        return self
    
    def add_meal(self, dish: str, quantity_in_grams: int) -> 'PlanBuilder': 
        return self
    
    @abc.abstractmethod
    def build(self) -> Plan:
        pass



class BodyBuildingPlanBuilder(PlanBuilder):
    
    def __init__(self, student: str):
        self.plan = Plan(student)

    @staticmethod
    def create_default_plan(student: str) -> Plan:
        return (BodyBuildingPlanBuilder(student)
                .add_day('Legs')
                    .add_meal('Cream cracker', 5)
                    .add_exercise('Polichinelo',15)
                .add_day('Arms')
                    .add_meal('Trakinas', 5)
                    .add_exercise('Flexao', 5)
                .add_day('Rest')
                    .add_meal('Big Mac', 2)
                .add_day('Cardio')
                    .add_meal('Pacoquita', 2)
                    .add_exercise('Volta no quarteirao', 2)
                .build())
    
    def add_meal(self, dish: str, quantity_in_grams: int) -> 'PlanBuilder':
        self.current_day().add_meal(Meal(dish, quantity_in_grams))
        return self
    
    def add_exercise(self, name: str, reps: int) -> 'PlanBuilder':
        self.current_day().add_exercise(Exercise(name, reps))
        return self
        
    def current_day(self) -> Day:
        return self.plan.days[-1]
    
    def build(self) -> Plan:
        return self.plan


default_plan = BodyBuildingPlanBuilder.create_default_plan('Joaozinho')

print(default_plan)


Joaozinho

Day 1: 
Objective: Legs
Exercises: 
	Polichinelo (x15)
Meals: 
	Cream cracker (5g)

Day 2: 
Objective: Arms
Exercises: 
	Flexao (x5)
Meals: 
	Trakinas (5g)

Day 3: 
Objective: Rest
Meals: 
	Big Mac (2g)

Day 4: 
Objective: Cardio
Exercises: 
	Volta no quarteirao (x2)
Meals: 
	Pacoquita (2g)


## Prós e contras:

### Prós:

- Isola as partes complexas do sistema
- Cria uma interface simplificada

### Contras:

- O _Facade_ em si, por depender de diversas partes do sistema, pode ser difícil de manter

## Discussao:

Part 1: How complex must a sub-system be in order to justify using a facade?
Part 2: What are the additional uses of a facade with respect to an organization of designers and developers with varying abilities? What are the political ramifications?
